<a href="https://colab.research.google.com/github/Alias23/IO-PRACTICAS/blob/main/IO_PRACTICA_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from pulp import (
    LpProblem, LpVariable, LpMinimize,
    lpSum, LpStatus, value, GLPK_CMD
)

# ----------------------------
# 1) Definir los datos
# ----------------------------
vert = ["A","B","C","D","E"]
n = len(vert)

# Oferta neta de cada nodo (b_i)
#   >0: nodo fuente (oferta)
#   <0: nodo sumidero (demanda)
#    0: nodo de transbordo
b = np.array([
    # A,  B,   C,   D,   E
      20, -15, -10,   5,   0
])

# Costes unitarios c_{ij} (n×n)
c = np.array([
    # A   B   C   D   E
    [  0,  2,  3,  0,  1],  # desde A
    [  2,  0,  1,  3,  0],  # desde B
    [  3,  1,  0,  2,  2],  # desde C
    [  0,  3,  2,  0,  1],  # desde D
    [  1,  0,  2,  1,  0],  # desde E
])

# Capacidades máximas u_{ij} (n×n). Si es ilimitada, pon un número grande.
u = np.array([
    # A   B   C   D   E
    [  0, 10, 10,  0,  5],
    [ 10,  0,  5, 10,  0],
    [ 10,  5,  0, 10, 10],
    [  0, 10, 10,  0,  5],
    [  5,  0, 10,  5,  0],
])

# ----------------------------
# 2) Construir el modelo PuLP
# ----------------------------
model = LpProblem("Flujo_MinCoste", LpMinimize)

# Variables x[i][j] ∈ [0, u_ij]
x = [
    [
        LpVariable(f"x_{i}_{j}",
                   lowBound=0,
                   upBound=u[i,j])
        for j in range(n)
    ]
    for i in range(n)
]

# Objetivo: minimizar ∑_{i,j} c_{ij} x_{ij}
model += lpSum(c[i,j] * x[i][j]
               for i in range(n)
               for j in range(n)), "Coste_Total"

# Balance de flujo en cada nodo: ∑_j x_{ij} – ∑_k x_{ki} = b_i
for i in range(n):
    model += (
        lpSum(x[i][j] for j in range(n))
        - lpSum(x[k][i] for k in range(n))
        == b[i],
        f"Balance_{vert[i]}"
    )

# ----------------------------
# 3) Resolver
# ----------------------------
model.solve(GLPK_CMD(msg=False))
# — o bien: model.solve() para usar CBC sin instalaciones extra

# ----------------------------
# 4) Mostrar resultados
# ----------------------------
print("Status:", LpStatus[model.status])
print("Coste mínimo:", value(model.objective))
print("Flujos óptimos:")
for i in range(n):
    for j in range(n):
        val = x[i][j].value()
        if val and val > 1e-6:
            print(f"  {vert[i]} → {vert[j]} : {val}")


Status: Optimal
Coste mínimo: 65.0
Flujos óptimos:
  A → B : 10.0
  A → C : 10.0
  D → B : 5.0
